In [3]:
import matplotlib
import scipy 
import sys
import argparse
import numpy
import dadi
from dadi import Numerics, PhiManip, Integration, Spectrum, Misc, Demographics1D
import dadi.Godambe

In [12]:
fs= dadi.Spectrum.from_file("GOC-30.sfs")
ns = fs.sample_sizes # get sample size from SFS (in haploids)
pts_l = [ns[0]+5,ns[0]+15,ns[0]+25] # this should be slightly larger (+5) than sample size and increase by 10
maxiter= 100

In [13]:
#Bootstraping
datafile = './SNPs_for_SFS_GOC.recode.vcf.gz'
dd = dadi.Misc.make_data_dict_vcf(datafile, './GOC_samples.txt')

In [14]:
# Generate 1000 bootstrap datasets, by dividing the genome into 2 Mb chunks and
# resampling from those chunks.
Nboot, chunk_size = 1000, 2e6
chunks = dadi.Misc.fragment_data_dict(dd, chunk_size)

In [15]:
pop_ids, ns = ['GOC'], [30]
boots = dadi.Misc.bootstraps_from_dd_chunks(chunks, Nboot, pop_ids, ns, polarized=False)

## 2 Epoch Model

In [14]:
func2 = Demographics1D.two_epoch
func_ex2 = dadi.Numerics.make_extrap_log_func(func)
popt2 = [0.354230329,0.227167841] 

In [15]:
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [4.67546275e-03 6.49583676e-03 1.04470115e+04]


In [16]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [6.37167352e-02 7.88030167e-02 1.39131933e+05]


## 3 Epoch Model

In [8]:
def bottleneck(params, ns, pts):
    nuB,nuF,TB,TF = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = Integration.one_pop(phi, xx, TB, nuB)  # bottleneck
    phi = Integration.one_pop(phi, xx, TF, nuF) # recovery
    fs = Spectrum.from_phi(phi, ns, (xx,))
    return fs
func=bottleneck
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [22]:
# nuB, nuF, TB, TF
popt = [0.153560308,0.000319266,0.307682258,2.93E-06] 
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [3.69178612e-03 8.42828602e-06 2.45267284e-03            nan
 4.48814258e+04]


In [23]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [5.80243398e-01 6.80698167e-04 5.44521879e-01 1.46301389e-05
 6.06228963e+06]


## 4 Epoch Model

In [24]:
def bottleneck(params, ns, pts):
    nuB,nuF,nuC,TB,TF,TC = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = Integration.one_pop(phi, xx, TB, nuB)  # bottleneck
    phi = Integration.one_pop(phi, xx, TF, nuF) # recovery
    phi = Integration.one_pop(phi, xx, TC, nuC) # current
    fs = Spectrum.from_phi(phi, ns, (xx,))
    return fs
func=bottleneck
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [25]:
#nuB,nuF,nuC,TB,TF,TC
popt = [0.00062526,3.401000298,0.006608544,0.000247327,0.078570568,0.00041039] 
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [           nan 2.68031888e-01            nan            nan
 6.02046707e-04            nan 7.29685467e+03]


In [26]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [8.24529632e-04 3.33637567e-01 2.37470968e-03 2.58735450e-04
 4.63534333e-03 1.12584382e-04 8.64509018e+04]


## LL ratio test

In [9]:
#simple model 2 Epoch model
func2 = Demographics1D.two_epoch
func_ex2 = dadi.Numerics.make_extrap_log_func(func2)
popt2 = [0.354230329,0.227167841] 
model2 = func_ex2(popt2, ns, pts_l)
ll_E2 = -1251.262324

In [10]:
# 3 EPoch model
func=bottleneck
func_ex = dadi.Numerics.make_extrap_log_func(func)
ll_E3 = -776.6687116

In [16]:
# 2 Epoch vs 3Epoch
p_lrt = [0.354230329,0,0.227167841,0]

adj = dadi.Godambe.LRT_adjust(func_ex, pts_l, boots, p_lrt, fs, 
                              nested_indices=[1,3], multinom=True)
D_adj = adj*2*(ll_E3 - ll_E2)
print('Adjusted D statistic: {0:.4f}'.format(D_adj))
pval = dadi.Godambe.sum_chi2_ppf(D_adj, weights=(0.5,0.5))
print('p-value for rejecting no-migration model: {0:.4f}'.format(pval))

ValueError: A population size is 0. Has the model been mis-specified?